In [1]:
import toolz as tz
execfile("./utils.py")

Some resources for the project

 - http://www.speech.cs.cmu.edu/cgi-bin/cmudict
 - http://www.nltk.org/book/ch11.html
 - https://en.wikipedia.org/wiki/Mnemonic_major_system
 - https://github.com/rhdunn/cmudict-tools

In [2]:
def parse_mnemonic_line(line):
    phoneme, value = line.split(" ")
    return (phoneme, value)

mnemonic_map = tz.thread_last(
    "./mnemonics.txt",
    read_lines,
    (map, parse_mnemonic_line),
    dict
)

mnemonic_phonemes = mnemonic_map.keys()
illegal_phonemes = [phoneme for phoneme, value in mnemonic_map.items() if value == "X"]
legal_phonemes = [phoneme for phoneme, value in mnemonic_map.items() if value != "X"]

In [3]:
def parse_cmu_line(line):
    chunks = line.split()
    word = strip_suffix(chunks[0], ["(1)", "(2)", "(3)", "(4)"])
    drop_numbers = lambda string: "".join([char for char in string if char not in "0123456789"])
    is_consonant = lambda phoneme: phoneme in mnemonic_phonemes
    
    all_phonemes = map(drop_numbers, chunks[1:]) # emphasis annotations aren't needed here
    phonemes = filter(is_consonant, all_phonemes) # vowels aren't part of the mnemonic
    numbers = [mnemonic_map[phoneme] for phoneme in phonemes]
    
    if (nvenn(phonemes, illegal_phonemes)[1] > 0) or (nvenn(phonemes, legal_phonemes)[1] < 1):
        return [] # ignore words with illegal phonemes, or which contain no allowed phonemes
    else: 
        return [{"word": word, "phonemes": phonemes, "all_phonemes": all_phonemes, "numbers": numbers}]
    
cmu = tz.thread_last(
    './cmudict.txt',
    read_file,
    split_lines,
    (filter, lambda line: not line.startswith(";;;")),
    (tz.mapcat, parse_cmu_line),
    list
)

In [4]:
def search_words(f): 
    return [entry for entry in cmu if f(entry['word'])]

def search_numbers(f): 
    return [entry['word'] for entry in cmu if f(entry['numbers'])]

def search_phonemes(f): 
    return [entry for entry in cmu if f(entry['phonemes'])]

In [5]:
def partition(n, seq):
    if len(seq) <= n:
        return [seq]
    else: 
        return [seq[:n]] + partition(n, seq[n:])
    
def translate(numbers):
    return tz.thread_last(
        numbers, 
        str,
        list,
        (partition, 3),
        (map, lambda query_numbers: search_numbers(lambda numbers: numbers == query_numbers)),
        list
    )

# Test it out!
Try converting some numbers into words

In [6]:
translate(123)

[['AUTONOMY',
  'AUTONOMY',
  'DENHAM',
  'DENIM',
  'DENOMME',
  'DONHAM',
  'DOWNHAM',
  'DOWNUM',
  'DUNHAM',
  'DUNNAM',
  'DYNAMO',
  'IDEONOMY']]

In [7]:
[x[0] for x in translate(54323432)]

['ALARM', 'ANAMARIA', 'AHMANN']

# Next steps


1. Find good metric for results. Should be easy to remember and visualize.
 - maybe use part of speech
 - manually select your favorite matches
 - best is probably to assess how common the word is (how? using other nltk corpus?)
 
 
If I only allow for 3-digit matches, then that 10^3 combinations to think about. So, there are around 1000 cases to handle. For each of these, it might be good to have several words to choose from so things are more interesting. Some manual work is conceivable.

In [8]:
# Try to get parts of speech
from nltk.corpus import wordnet as wn
wn.synsets('HEAD')[:5]
[x.pos() for x in wn.synsets('HEAD')[:5]]

[u'n', u'n', u'n', u'n', u'n']

## Word frequencies
This could help pick the best match from a set of options. 
 - http://subtlexus.lexique.org/
 - http://www.natcorp.ox.ac.uk/